In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [29]:
import pandas as pd
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', 10000)
import numpy as np
import string
import random
from SoundsLike.SoundsLike import Search
import Levenshtein

ModuleNotFoundError: No module named 'SoundsLike'

In [9]:
df = pd.read_csv("../../data/conll03/train.txt", sep=" ")
#df = pd.read_csv("data/dev.txt", sep=" ")
#df = pd.read_csv("data/test.txt", sep=" ")

In [10]:
df.head()

,-DOCSTART-,-X-,-X-.1,O
0,EU,NNP,B-NP,B-ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,B-MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O


In [11]:
df = df[df['O'].notna()]

In [12]:
df.drop(['-X-', '-X-.1'], axis=1, inplace=True)

In [13]:
df.columns = ['Word', 'Tag']

In [14]:
df.head()

,Word,Tag
0,EU,B-ORG
1,rejects,O
2,German,B-MISC
3,call,O
4,to,O


In [15]:
#df.drop(df[df['Word'] == "-DOCSTART-"].index, axis=0, inplace=True)

In [16]:
df['Tag'].unique()

array(['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG',
       'I-MISC', 'I-LOC'], dtype=object)

In [17]:
tags = df['Tag'].values
words = df['Word'].values

In [18]:
new_tags = []
for t in tags:
    if t == "B-ORG":
        new_tags.append("ORG")
    elif t == "B-PER":
        new_tags.append("PER")
    elif t == "I-PER":
        new_tags.append("PER")
    elif t == "B-LOC":
        new_tags.append("LOC")
    elif t == "I-ORG":
        new_tags.append("ORG")
    elif t == "I-LOC":
        new_tags.append("LOC")
    else:
        new_tags.append("O")

In [19]:
df.drop(['Tag'], axis=1, inplace=True)
df.drop(['Word'], axis=1, inplace=True)

In [20]:
df['Tag'] = new_tags
df['Word'] = words

In [21]:
len(df)

202388

In [22]:
sentence_no = 0
sentence = []
for w in words:
    sentence.append(sentence_no)
    if w == ".":
        sentence_no = sentence_no + 1

In [23]:
df.insert(0, 'Sentence #', sentence)

In [24]:
df.head()

,Sentence #,Tag,Word
0,0,ORG,EU
1,0,O,rejects
2,0,O,German
3,0,O,call
4,0,O,to


In [25]:
df['Tag'].unique()

array(['ORG', 'O', 'PER', 'LOC'], dtype=object)

In [26]:
len(df)

202388

In [27]:
g_test = df.groupby("Sentence #")
test_df = pd.DataFrame({"Sentence": g_test.apply(lambda sdf: " ".join(map(str,sdf.Word))),
                       "Tag": g_test.apply(lambda sdf: ",".join(sdf.Tag))})
test_df.reset_index(inplace=True)

In [32]:
test_df.head(100)

,Sentence #,Sentence,Tag
0,0,EU rejects German call to boycott British lamb .,"ORG,O,O,O,O,O,O,O,O"
1,1,Peter Blackburn BRUSSELS 1996-08-22 The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .,"PER,PER,LOC,O,O,ORG,ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,2,Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .,"LOC,O,O,O,O,ORG,ORG,O,O,O,PER,PER,O,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O,O"
3,3,"We do n't support any such recommendation because we do n't see any grounds for it , the Commission 's chief spokesman Nikolaus van der Pas told a news briefing .","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,O,O,O,PER,PER,PER,PER,O,O,O,O,O"
4,4,He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,ORG,O"
5,5,"He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health .","O,O,O,O,O,O,O,ORG,O,O,PER,PER,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
6,6,Fischler proposed EU-wide measures after reports from Britain and France that under laboratory conditions sheep could contract Bovine Spongiform Encephalopathy ( BSE ) -- mad cow disease .,"PER,O,O,O,O,O,O,LOC,O,LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
7,7,"But Fischler agreed to review his proposal after the EU 's standing veterinary committee , mational animal health officials , questioned if such action was justified as there was only a slight risk to human health .","O,PER,O,O,O,O,O,O,O,ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
8,8,Spanish Farm Minister Loyola de Palacio had earlier accused Fischler at an EU farm ministers ' meeting of causing unjustified alarm through dangerous generalisation .,"O,O,O,PER,PER,PER,O,O,O,PER,O,O,ORG,O,O,O,O,O,O,O,O,O,O,O,O"
9,9,.,O


In [22]:
dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if tag != "O":
        error = []
        try:
            x = Search.perfectHomophones(word.lower())
            for d in x:
                err = Levenshtein.distance(word.lower(), d)
                error.append(err)
                word = x[error.index(max(error))]
        except ValueError:
            pass 
        dataset.append((sentence_no, tag, word))
    else:
        dataset.append((sentence_no, tag, word))
df = pd.DataFrame(dataset, columns=['Sentence #', 'Tag', 'Word'])

In [ ]:
df.head(100)

In [ ]:
df.to_csv("../../data/conll03/new_conll_train_preprocessed_with_punctuation_with_broken_entities2.csv")

In [ ]:
entities = df[df['Tag'] != "O"]['Sentence #'].values.tolist()

In [ ]:
per_entities = df[df['Tag'] == "PER"]['Sentence #'].values.tolist()

In [ ]:
per_randomlist = random.sample(per_entities, int(len(per_entities)/2))
per_randomlist.sort()

In [ ]:
len(per_randomlist)

In [ ]:
per_dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in per_randomlist and tag == "PER":
        word = list(word)
        ind = random.randint(0,len(word)-1)
        temp = word[ind-1]
        word[ind-1] = word[ind]
        word[ind] = temp
        word = "".join(word)
        per_dataset.append((sentence_no, tag, word))
    else:
        per_dataset.append((sentence_no, tag, word))
df = pd.DataFrame(per_dataset, columns=['Sentence #', 'Tag', 'Word'])

In [ ]:
df.head(10)

In [ ]:
loc_entities = df[df['Tag'] == "LOC"]['Sentence #'].values.tolist()

In [ ]:
loc_randomlist = random.sample(loc_entities, int(len(loc_entities)/2))
loc_randomlist.sort()

In [ ]:
loc_dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in loc_randomlist and tag == "LOC":
        word = list(word)
        ind = random.randint(0,len(word)-1)
        temp = word[ind-1]
        word[ind-1] = word[ind]
        word[ind] = temp
        word = "".join(word)
        loc_dataset.append((sentence_no, tag, word))
    else:
        loc_dataset.append((sentence_no, tag, word))
df = pd.DataFrame(loc_dataset, columns=['Sentence #', 'Tag', 'Word'])

In [ ]:
org_entities = df[df['Tag'] == "ORG"]['Sentence #'].values.tolist()

In [ ]:
org_randomlist = random.sample(org_entities, int(len(org_entities)/2))
org_randomlist.sort()

In [ ]:
org_dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in org_randomlist and tag == "ORG":
        word = list(word)
        ind = random.randint(0,len(word)-1)
        temp = word[ind-1]
        word[ind-1] = word[ind]
        word[ind] = temp
        word = "".join(word)
        org_dataset.append((sentence_no, tag, word))
    else:
        org_dataset.append((sentence_no, tag, word))
df = pd.DataFrame(org_dataset, columns=['Sentence #', 'Tag', 'Word'])

In [ ]:
randomlist = random.sample(entities, int(len(entities)/2))
randomlist.sort()

In [ ]:
randomlist

In [ ]:
dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in randomlist:
        if tag != "O":
            if len(word) >=4:
                word = list(word)
                ind = random.randint(2,len(word)-1)
                temp = word[ind-1]
                word[ind-1] = word[ind]
                word[ind] = temp
                word = "".join(word)
                dataset.append((sentence_no, tag, word))
            else:
                dataset.append((sentence_no, tag, word))
        else:
            dataset.append((sentence_no, tag, word))
    else:
        dataset.append((sentence_no, tag, word))

In [ ]:
len(dataset)

In [ ]:
new_df = pd.DataFrame(dataset, columns=['Sentence #', 'Tag', 'Word'])

In [ ]:
df.head(100)

In [ ]:
df.to_csv("../../data/conll03/new_conll_train_preprocessed_with_punctuation_with_broken_entities1.csv")

In [50]:
#df = pd.read_csv("../../data/conll03/new_conll_train_preprocessed_with_punctuation_with_broken_entities1.csv")
df = pd.read_csv("../../data/conll03/new_conll_train_preprocessed_without_punctuation_with_broken_entities2.csv")

In [51]:
df.head()

,Unnamed: 0,Sentence #,Word,Tag
0,0,0,Eu,ORG
1,1,0,rejects,O
2,2,0,German,O
3,3,0,call,O
4,4,0,to,O


In [52]:
g_test = df.groupby("Sentence #")
test_df = pd.DataFrame({"Sentence": g_test.apply(lambda sdf: " ".join(map(str,sdf.Word))),
                       "Tag": g_test.apply(lambda sdf: ",".join(sdf.Tag))})
test_df.reset_index(inplace=True)

In [53]:
test_df.head(100)

,Sentence #,Sentence,Tag
0,0,Eu rejects German call to boycott British lamb,"ORG,O,O,O,O,O,O,O"
1,1,Pieter Blackburn Brussels 19960822 The European Commision said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep,"PER,PER,LOC,O,O,ORG,ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,2,Germanys representative to the European Unions veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Brittain until the scientific advice was clearer,"LOC,O,O,O,ORG,ORG,O,O,PER,PER,O,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O"
3,3,We dont support any such recommendation because we dont see any grounds for it the Commisions chief spokesman Nicolaus Vann Dirr Paz told a news briefing,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,O,O,PER,PER,PER,PER,O,O,O,O"
4,4,He said further scientific study was required and if it was found that action was needed it should be taken by the European Union,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,ORG"
5,5,He said a proposal last month by Eu Farm Commissioner Franze Fischler to ban sheep brains spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health,"O,O,O,O,O,O,O,ORG,O,O,PER,PER,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
6,6,Fischler proposed EUwide measures after reports from Brittain and France that under laboratory conditions sheep could contract Bovine Spongiform Encephalopathy BSE mad cow disease,"PER,O,O,O,O,O,O,LOC,O,LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
7,7,But Fischler agreed to review his proposal after the Eus standing veterinary committee mational animal health officials questioned if such action was justified as there was only a slight risk to human health,"O,PER,O,O,O,O,O,O,O,ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
8,8,Spanish Farm Minister Loyola Du Palacio had earlier accused Fischler at an Eu farm ministers meeting of causing unjustified alarm through dangerous generalisation,"O,O,O,PER,PER,PER,O,O,O,PER,O,O,ORG,O,O,O,O,O,O,O,O,O,O"
9,10,Only France and Brittain backed Fischlers proposal,"O,LOC,O,LOC,O,PER,O"
